In [1]:
%load_ext autoreload

In [11]:
import sys
import os

In [12]:
os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['OPENAI_API_KEY'] = 'zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
sys.path.append('./src')

In [13]:
%autoreload

import json
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from simpletransformers.classification import ClassificationModel

import tqdm
import random
import datasets

In [10]:
model = ClassificationModel("xlmroberta", "classla/xlm-roberta-base-multilingual-text-genre-classifier", use_cuda=True)

In [ ]:
predictions, logit_output = model.predict(["How to create a good text classification model? First step is to prepare good data. Make sure not to skip the exploratory data analysis. Pre-process the text if necessary for the task. The next step is to perform hyperparameter search to find the optimum hyperparameters. After fine-tuning the model, you should look into the predictions and analyze the model's performance. You might want to perform the post-processing of data as well and keep only reliable predictions.", 
                                        "On our site, you can find a great genre identification model which you can use for thousands of different tasks. With our model, you can fastly and reliably obtain high-quality genre predictions and explore which genres exist in your corpora. Available for free!"]
                                        )
predictions
# Output: array([3, 8])

[model.config.id2label[i] for i in predictions]
# Output: ['Instruction', 'Promotion']


In [ ]:
df_paths = Path(ds_path).glob("{}*.jsonl".format(split))
df_paths = [path for path in df_paths if 'filtered' not in str(path)]
print(df_paths)

dfs = {p: pd.read_json(p, lines=True) for p in df_paths}


In [18]:
dataset = datasets.load_dataset("json", data_files=["/mnt/swordfish-pool2/milad/hiatus-data/performers-data/tmp-data/dev_queries.jsonl"])

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['documentID', 'authorIDs', 'fullText', 'spanAttribution', 'collectionNum', 'source', 'dateCollected', 'publiclyAvailable', 'deidentified', 'languages', 'lengthWords', 'dateCreated', 'timeCreated', 'isForeground'],
        num_rows: 33538
    })
})

In [5]:
def author_pair_similarity(documents_ids, documents):
    #print(documents)
    document_embeddings = model.encode(documents)
    #print(document_embeddings)
    pairwise_sim = cosine_similarity(document_embeddings, dense_output=True)
    #print(pairwise_sim)
    docs_sims = {}
    for i, di in enumerate(documents_ids):
        for j, dj in enumerate(documents_ids):
            if j <= i:
                continue
            docs_sims[(di, dj)] = pairwise_sim[i,j]
            
    return docs_sims

In [7]:
#ds_path = '/mnt/swordfish-pool2/milad/hiatus-data/performers-data/tmp-data/'
ds_path = '/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/'
df_info = extract_split_similarity_info(ds_path, 'dev', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'test', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'train', max_sample_size=50000)

[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/dev_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/dev_queries.jsonl')]


33538it [03:46, 147.76it/s]


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/dev_candidates.jsonl 33538 --> 5000
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/dev_queries.jsonl 33538 --> 5000
[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/test_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/test_queries.jsonl')]


65102it [06:54, 157.21it/s]


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/test_candidates.jsonl 65102 --> 5000
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/test_queries.jsonl 65103 --> 5000
[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/train_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/train_queries.jsonl')]


558885it [59:37, 156.24it/s]


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/train_candidates.jsonl 558926 --> 50000
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/gmane/train_queries.jsonl 558918 --> 50000


In [8]:
#ds_path = '/mnt/swordfish-pool2/milad/hiatus-data/performers-data/tmp-data/'
ds_path = '/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/'
df_info = extract_split_similarity_info(ds_path, 'dev', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'test', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'train', max_sample_size=50000)

[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/dev_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/dev_queries.jsonl')]


3379it [00:29, 116.21it/s]


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/dev_candidates.jsonl 3379 --> 437
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/dev_queries.jsonl 3379 --> 437
[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/test_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/test_queries.jsonl')]


6560it [00:56, 116.87it/s]


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/test_candidates.jsonl 6560 --> 891
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/test_queries.jsonl 6560 --> 891
[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/train_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/train_queries.jsonl')]


56319it [08:05, 115.93it/s]


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/train_candidates.jsonl 56319 --> 7469
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/bookcorpus/train_queries.jsonl 56319 --> 7469


In [9]:
#ds_path = '/mnt/swordfish-pool2/milad/hiatus-data/performers-data/tmp-data/'
ds_path = '/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews'
df_info = extract_split_similarity_info(ds_path, 'dev', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'test', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'train', max_sample_size=50000)

[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/dev_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/dev_queries.jsonl')]


10861it [01:48, 100.14it/s]


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/dev_candidates.jsonl 10861 --> 2847
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/dev_queries.jsonl 10861 --> 2847
[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/test_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/test_queries.jsonl')]


21085it [03:33, 98.56it/s] 


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/test_candidates.jsonl 21085 --> 5000
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/test_queries.jsonl 21085 --> 5000
[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/train_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/train_queries.jsonl')]


181024it [30:11, 99.92it/s] 


/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/train_candidates.jsonl 181024 --> 46617
/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/realnews/train_queries.jsonl 181024 --> 46617


In [ ]:
#ds_path = '/mnt/swordfish-pool2/milad/hiatus-data/performers-data/tmp-data/'
ds_path = '/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/ao3/'
df_info = extract_split_similarity_info(ds_path, 'dev', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'test', max_sample_size=5000)
df_info = extract_split_similarity_info(ds_path, 'train', max_sample_size=25000)

[PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/ao3/dev_candidates.jsonl'), PosixPath('/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/ao3/dev_queries.jsonl')]


In [1]:
# from datasets import Dataset, load_dataset
# ao3 = load_dataset('json', data_files={'train': ['/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/ao3/train_queries.jsonl', '/burg/old_dsi/users/ma4608/hiatus_performers_data/sadiri/ao3/train_candidates.jsonl'],})
# ao3_sample = ao3['train']#.select(range(10))
# ao3_sample = ao3_sample.map(lambda row: {'authorID': row['authorIDs'][0]})

# import polars as pl
# polars_df = ao3_sample.to_polars()
# polars_df = polars_df.group_by('authorID').agg(pl.col("fullText"))